In [3]:
# libraries

from datetime import datetime
import os
import glob
import requests 
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Web scrapping

In [4]:
# download data 
# =============

link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSc_2y5N0I67wDU38DjDh35IZSIS30rQf7_NYZhtYYGU1jJYT6_kDx4YpF-qw0LSlGsBYP8pqM_a1Pd/pubhtml#'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [24]:
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

# print(tbody)
# print(body)

In [25]:
head_row = [i.text for i in body[0].find_all('td')]
head_row

['Patient Number',
 'State Patient Number',
 'Date Announced',
 'Age Bracket',
 'Gender',
 'Detected City',
 'Detected District',
 'Detected State',
 'State_code',
 'Current Status',
 'Notes',
 'Contracted from which Patient (Suspected)',
 'Nationality',
 'Type of transmission',
 'Status Change Date',
 'Source_1',
 'Source_2',
 'Source_3',
 'Backup Notes',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [26]:
contents = []

for i in range(len(body)):
    contents.append([i.text for i in body[i].find_all('td')])

# Saving to Dataframe

In [27]:
p_df = pd.DataFrame(contents[2:len(contents)], columns=head_row)
p_df.head()

,Patient Number,State Patient Number,Date Announced,Age Bracket,Gender,Detected City,Detected District,Detected State,State_code,Current Status,...,Source_2,Source_3,Backup Notes,,,,,,,
0,1,KL-TS-P1,30/01/2020,20,F,Thrissur,Thrissur,Kerala,KL,Recovered,...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,,,,,,,,None
1,2,KL-AL-P1,02/02/2020,,,Alappuzha,Alappuzha,Kerala,KL,Recovered,...,https://weather.com/en-IN/india/news/news/2020...,,Student from Wuhan,,,,,,,
2,3,KL-KS-P1,03/02/2020,,,Kasaragod,Kasaragod,Kerala,KL,Recovered,...,https://twitter.com/ANI/status/122422148580539...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,,,,,,,
3,4,DL-P1,02/03/2020,45,M,East Delhi (Mayur Vihar),East Delhi,Delhi,DL,Recovered,...,https://economictimes.indiatimes.com/news/poli...,,Travel history to Italy and Austria,,,,,,,
4,5,TS-P1,02/03/2020,24,M,Hyderabad,Hyderabad,Telangana,TG,Recovered,...,https://www.indiatoday.in/india/story/coronavi...,https://www.thehindu.com/news/national/coronav...,"Travel history to Dubai, Singapore contact",,,,,,,


# Data cleaning and transformations

In [28]:
# shape of dataframe
# ==================

p_df.shape

(3066, 26)

In [29]:
# columns
# =======

p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'State_code', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Type of transmission', 'Status Change Date', 'Source_1', 'Source_2',
       'Source_3', 'Backup Notes', '', '', '', '', '', '', ''],
      dtype='object')

In [30]:
# selecting important columns only
# ================================

p_df = p_df.loc[:, :'Backup Notes']

In [31]:
# looking for missing values 
# ==========================

p_df.isna().sum()

Patient Number                               0
State Patient Number                         0
Date Announced                               0
Age Bracket                                  0
Gender                                       0
Detected City                                0
Detected District                            0
Detected State                               0
State_code                                   0
Current Status                               0
Notes                                        0
Contracted from which Patient (Suspected)    0
Nationality                                  0
Type of transmission                         0
Status Change Date                           0
Source_1                                     0
Source_2                                     0
Source_3                                     0
Backup Notes                                 0
dtype: int64

In [32]:
# no. of empty strings in each column
# ===================================

print(p_df.shape)

for i in p_df.columns:
    print(i, '\t', p_df[p_df[i]==''].shape[0])

(3066, 19)
Patient Number 	 54
State Patient Number 	 2445
Date Announced 	 419
Age Bracket 	 2291
Gender 	 2139
Detected City 	 2247
Detected District 	 1123
Detected State 	 419
State_code 	 427
Current Status 	 426
Notes 	 797
Contracted from which Patient (Suspected) 	 2354
Nationality 	 2641
Type of transmission 	 1302
Status Change Date 	 421
Source_1 	 437
Source_2 	 2110
Source_3 	 2903
Backup Notes 	 2706


In [33]:
# replacing empty strings with np.nan
# ==================================-

print(p_df.shape)

p_df = p_df.replace(r'', np.nan, regex=True)
p_df.isna().sum()

(3066, 19)


Patient Number                                 54
State Patient Number                         2445
Date Announced                                419
Age Bracket                                  2291
Gender                                       2139
Detected City                                2247
Detected District                            1123
Detected State                                419
State_code                                    427
Current Status                                426
Notes                                         797
Contracted from which Patient (Suspected)    2354
Nationality                                  2641
Type of transmission                         1302
Status Change Date                            421
Source_1                                      437
Source_2                                     2110
Source_3                                     2903
Backup Notes                                 2706
dtype: int64

In [34]:
# droping empty rows (row with just row number but without patient entry
# ======================================================================

p_df.dropna(subset=['Detected State'], inplace=True)
print(p_df.shape)
p_df.isna().sum()

(2647, 19)


Patient Number                                  0
State Patient Number                         2026
Date Announced                                  0
Age Bracket                                  1872
Gender                                       1720
Detected City                                1828
Detected District                             704
Detected State                                  0
State_code                                      8
Current Status                                  7
Notes                                         378
Contracted from which Patient (Suspected)    1935
Nationality                                  2222
Type of transmission                          883
Status Change Date                              3
Source_1                                       18
Source_2                                     1691
Source_3                                     2484
Backup Notes                                 2287
dtype: int64

In [35]:
p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'State_code', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Type of transmission', 'Status Change Date', 'Source_1', 'Source_2',
       'Source_3', 'Backup Notes'],
      dtype='object')

In [36]:
# rename dateframe columns 
# ========================

p_df.columns = ['_'.join(col.lower().split()) for col in p_df.columns]
p_df.rename(columns = {'contracted_from_which_patient_(suspected)':'suspected_contacted_patient'}, inplace=True)
p_df.sample(5)

,patient_number,state_patient_number,date_announced,age_bracket,gender,detected_city,detected_district,detected_state,state_code,current_status,notes,suspected_contacted_patient,nationality,type_of_transmission,status_change_date,source_1,source_2,source_3,backup_notes
1782,1783,TN-P171,01/04/2020,39,M,Mettupalayam,Coimbatore,Tamil Nadu,TN,Hospitalized,Attended Delhi Religious Conference,E0,NaN,NaN,01/04/2020,https://twitter.com/NHM_TN/status/124533236456...,NaN,NaN,NaN
2518,2519,NaN,02/04/2020,NaN,NaN,NaN,NaN,Andhra Pradesh,AP,Hospitalized,NaN,NaN,NaN,NaN,02/04/2020,https://twitter.com/ArogyaAndhra/status/124575...,NaN,NaN,NaN
2139,2140,NaN,02/04/2020,NaN,NaN,NaN,NaN,Delhi,DL,Hospitalized,NaN,NaN,NaN,NaN,02/04/2020,https://twitter.com/ANI/status/124568322055053...,NaN,NaN,NaN
431,432,NaN,23/03/2020,NaN,NaN,Delhi,NaN,Delhi,DL,Hospitalized,Details awaited,P171,NaN,TBD,23/03/2020,https://twitter.com/ArvindKejriwal/status/1241...,https://twitter.com/ANI/status/124196678221620...,NaN,NaN
1353,1354,NaN,31/03/2020,17,F,NaN,Ajmer,Rajasthan,RJ,Hospitalized,"Contact of Patient, ID Unknown",NaN,NaN,TBD,31/03/2020,https://twitter.com/ANI/status/124483610787688...,NaN,NaN,NaN


In [37]:
# creating patient id column from patient number
# ===============================================

p_df['p_id'] = p_df['patient_number'].apply(lambda x : 'P'+str(x))
p_df.columns

Index(['patient_number', 'state_patient_number', 'date_announced',
       'age_bracket', 'gender', 'detected_city', 'detected_district',
       'detected_state', 'state_code', 'current_status', 'notes',
       'suspected_contacted_patient', 'nationality', 'type_of_transmission',
       'status_change_date', 'source_1', 'source_2', 'source_3',
       'backup_notes', 'p_id'],
      dtype='object')

In [38]:
p_df = p_df.loc[:, :'backup_notes']

# Saving Data

In [39]:
p_df.sample(5)

,patient_number,state_patient_number,date_announced,age_bracket,gender,detected_city,detected_district,detected_state,state_code,current_status,notes,suspected_contacted_patient,nationality,type_of_transmission,status_change_date,source_1,source_2,source_3,backup_notes
422,423,NaN,23/03/2020,27,F,Vadodara,Vadodara,Gujarat,GJ,Hospitalized,Daughter of P269,P269,India,Local,23/03/2020,https://twitter.com/PTI_News/status/1241956921...,https://indianexpress.com/article/india/gujara...,NaN,NaN
256,257,NaN,20/03/2020,53,M,Kalyan,Thane,Maharashtra,MH,Hospitalized,Travelled from Dubai,NaN,NaN,Imported,20/03/2020,https://www.thehindu.com/news/cities/mumbai/fi...,NaN,NaN,Travel History to Dubai
953,954,KA-P75,28/03/2020,20,M,Davangere,Davanagere,Karnataka,KA,Hospitalized,Cousin of KA-P42,P629,NaN,Local,28/03/2020,https://karunadu.karnataka.gov.in/hfw/kannada/...,NaN,NaN,NaN
1533,1534,TN-P98,31/03/2020,NaN,M,NaN,Tirunelveli,Tamil Nadu,TN,Hospitalized,Attended Delhi Religious Conference,E0,NaN,TBD,31/03/2020,https://twitter.com/Vijayabaskarofl/status/124...,NaN,NaN,NaN
1025,1026,NaN,28/03/2020,48,M,NaN,Indore,Madhya Pradesh,MP,Hospitalized,contacts of earlier positive cases of #COVID19...,NaN,NaN,Local,28/03/2020,https://twitter.com/ANI/status/124399916468745...,https://twitter.com/ANI/status/124399928428384...,NaN,NaN


In [40]:
p_df.to_csv('patients_data.csv', index=False)